## dataprep
reformat dataset for use with TensorFlow and API

In [ ]:
!pip install pygithub
!wget https://github.com/PubChimps/think2019/blob/master/data/cloudclassifier.npy.zip?raw=true
!mv cloud* clouddata.npz.zip
!unzip clouddata.npz.zip

In [ ]:
import re
import nltk 
import numpy as np
import pandas as pd

nltk.download('punkt')

In [ ]:
data = np.load('./cloudclassifier.npy')

#example data
print(data[0])

In [ ]:
np.random.shuffle(data)
trainset, testset = np.array(data)[:int(len(data)*.8),:], np.array(data)[int(len(data)*.8):,:]

Build out dataset for neural network. Data needs to be tokenized and dictionary collected for bag of words encoding

In [ ]:
words = []

labeledlines = []
ignore_words = ['?', ',']

for line in trainset:
    text = str(re.split(r'[.,]', line[0])).replace("'","").replace('[','')
    text = re.sub(r'\b\w{1,1}\b', '', text)
    w = nltk.word_tokenize(text)
    w = [ele for ele in w if ele not in ignore_words]
    words.extend(w)
    labeledlines.append([w, line[1]])
    
testlines = []

for line in testset:
    text = str(re.split(r'[.,]', line[0])).replace("'","").replace('[','')
    text = re.sub(r'\b\w{1,1}\b', '', text)
    w = nltk.word_tokenize(text)
    w = [ele for ele in w if ele not in ignore_words]
    words.extend(w)
    testlines.append([w, line[1]])
    

words = list(set(words))

print('each data example will be transformed into a feature vector of size')
print(len(words))

### Translate feature vector via bag of words encoding and label into one-hot encoding

In [ ]:
traindata = []

for line in labeledlines:
    bag = []
    code = line[0]
    for w in words:
        bag.append(code.count(w)) 
        
    classes = [0,0,0,0]
    if line[1] == 'amazon':
        classes[0] = 1
    elif line[1] == 'ibm':
        classes[1] = 1
    elif line[1] == 'microsoft':
        classes[2] = 1
    elif line[1] == 'google':
        classes[3] = 1
    else:
        print(line[1])

    traindata.append([bag,classes])
    
testdata = []
for line in testlines:
    bag = []
    code = line[0]
    for w in words:
        bag.append(code.count(w)) 
        
    classes = [0,0,0,0]
    if line[1] == 'amazon':
        classes[0] = 1
    elif line[1] == 'ibm':
        classes[1] = 1
    elif line[1] == 'microsoft':
        classes[2] = 1
    elif line[1] == 'google':
        classes[3] = 1
    else:
        print(line[1])

    testdata.append([bag,classes])
    
x_train = np.array([row[0] for row in traindata]).T
y_train = np.array([row[1] for row in traindata]).T
x_test = np.array([row[0] for row in testdata]).T
y_test = np.array([row[1] for row in testdata]).T

np.save('x_train.npy',x_train)
np.save('y_train.npy',y_train)
np.save('x_test.npy',x_test)
np.save('y_test.npy',y_test)
np.save('testset.npy',testset)

Convert dataset to .csv file for API

In [ ]:
d = []
for i in labeledlines:
    for j in re.findall('.{1,1024}', str(i[0]).replace(',', ' ').replace("'",'').replace('[','').replace(']','')):
        d.append({'text': j, 'cloud': i[1]})
        
df = pd.DataFrame(d, columns = ['text', 'cloud'])
df['text'].replace(' ', np.nan, inplace=True)
df = df.dropna()
df.to_csv('cloudtrainingdata.csv', header=False,index=False)

List files via !ls 
There should be a total of eight

In [ ]:
!ls